In [344]:
import boto3
from langchain.vectorstores import Chroma
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.chains.conversation.memory import  ConversationSummaryMemory, ConversationBufferMemory
from langchain.agents import load_tools,Tool
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate

In [345]:
bedrock_embedding_model_id = 'amazon.titan-embed-text-v1'
model_llm_id = "anthropic.claude-instant-v1"
bedrock_client = boto3.client("bedrock-runtime")
model_parameter = {"temperature": 0.0, "top_p": .9, "max_tokens_to_sample": 2000}
llm = Bedrock(model_id=model_llm_id, model_kwargs=model_parameter,client=bedrock_client)

#table_name = os.environ.get("TABLE_NAME")

In [346]:
persist_directory = './qa-schedule-an-appointment/vectordb/docs/chroma/'

In [347]:
def create_langchain_vector_embedding_using_bedrock(bedrock_client, bedrock_embedding_model_id):
    bedrock_embeddings_client = BedrockEmbeddings(
        client=bedrock_client,
        model_id=bedrock_embedding_model_id)
    return bedrock_embeddings_client

In [348]:
bedrock_embeddings_client = create_langchain_vector_embedding_using_bedrock(bedrock_client, bedrock_embedding_model_id) 


In [349]:
def create_langchain_vector_embedding_using_bedrock(bedrock_client, bedrock_embedding_model_id):
    bedrock_embeddings_client = BedrockEmbeddings(
        client=bedrock_client,
        model_id=bedrock_embedding_model_id)
    return bedrock_embeddings_client

In [350]:
def build_prompt():
    prompt_template = """Human: Are you a spa assistant specializing in massages, you are very talkative and provides specific details so that the human can decide what answer is best for him.
    Use the following format:
        Question: the input question you must answer
    
    Assistant: OK, got it, I'll be a talkative truthful AI assistant.

    Human: Search this question in your documents: {question}

    Assistant: Here are some documents in <documents> tags that refer to what you are talking about :
    <documents>
    {context}
    </documents>
    Human: 
    Based on the above documents, give me detailed information so I can decide about: {question}.
    Answer "don't know" if documents are empty. 
    DON'T INVENT HYPOTHETICAL ANSWERS, uses ONLY metadata in {context}. 
    Final Answer: always reply in the original user language and human legible.

    Helpful Answer:"""

    updated_prompt = PromptTemplate(
    input_variables=[ 'question', 'context'], template=prompt_template)

    return updated_prompt

In [351]:
def Retrieval_QA(updated_prompt,vectordb):
    qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": updated_prompt},
    verbose=True,   
)
    return qa_chain

In [56]:
prompt = "que masaje me recomiendas para el dolor de espalda?"
vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=bedrock_embeddings_client
    )

massage_data = Retrieval_QA(build_prompt(),vectordb)
completion  = massage_data({"query": prompt})

print(completion['result'])



> Entering new RetrievalQA chain...

> Finished chain.
 Según los documentos, para el dolor de espalda te recomiendo el masaje miofascial. Este masaje se enfoca en los músculos, tendones y tejidos conectivos. Es adecuado para dolores musculares en la espalda causados por exceso de ejercicio, lesiones deportivas, mala postura o trabajo estático donde los músculos permanecen tensos por largos períodos. El masaje miofascial ayuda a aliviar las contracturas y tensiones musculares, mejorar la circulación sanguínea en la zona y la flexibilidad de los tejidos. De esta forma, alivia el dolor y la tensión en la espalda. Es un tipo de masaje suave pero profundo que puede ayudar a tratar y prevenir dolores de espalda recurrentes.


In [65]:
def get_massange_type(prompt):
    completion = Retrieval_QA(build_prompt(),vectordb)({"query": prompt})
    return completion['result']

In [441]:
import json
lambda_client = boto3.client('lambda')


def lambda_invoke (VALUE):
    LAMBDA_NAME = "QaScheduleAnAppointmentSt-FnDynamoDBputitem1CE3F31-0F9HoAYZvOq8"
    response = lambda_client.invoke(
                FunctionName = LAMBDA_NAME,
                InvocationType = 'Event' ,#'RequestResponse', 
                Payload = json.dumps({'prompt':VALUE})
            )

    print(f'\nRespuesta:{response}')

In [442]:
# Tool to put item in amazon dynamodb table
tools = []

In [443]:
lambda_query_function_name = "QaScheduleAnAppointmentSt-Fnquerydynamodbpassanger-UqcpPbg80Syo"
 
tools = load_tools(
    ["awslambda"],
    awslambda_tool_name="get_appointment_information",
    awslambda_tool_description="useful when you want to get appointment information data by phone number",
    function_name=lambda_query_function_name,
                    )


In [444]:
description_lambda_put_item = """useful when you have data to schedule an appointment in the spa agenda by date, time, massage_type, name, phone_number in json format """ 
tools.append(
    Tool.from_function(
        func=lambda_invoke,
        name="schedule_an_appointment",
        description=description_lambda_put_item
    ))

In [445]:
tools.append(
    Tool.from_function(
        func=get_massange_type,
        name="search_massage_type",
        description="Useful when you want to know about best massage"
    ))

In [498]:
def promp_definition():

    prompt_template = """
        You are a spa assistant who manages massage schedule/appointment. For that you have 5 tasks,each task is activated according to the user's input: 
        Task 1: Recommend the best massage. Recommend the best massage according to the user's preferences using ["search_massage_type"].
        Task 2: Gather user information. If the user wants to schedule, ask for the following information to the user: 
        date, time, name, massage_type, phone_number.
        PLEASE don't make up data, ask the user for missing information. 
        Task 3: Save appointment. Once you have the user schedule data, schedule appointment for the user using ["schedule_an_appointment"]. To schedule, follow the instructions encloset by the tag <instructions>.
        <instructions> 
        - the types of massages are: abdominal, cranial-facial, foot, lymphatic, perineal, respiratory-physiotherapy, scar, unloading-sports. 
        - Send the schedule data, date in YYYY-MM-DD format and time in HH:MM format, and in json format to ["schedule_an_appointment"].
        </instructions> 
        Task 4: Get appointmet information. For complete this task you need the user phone number, PLEASE don't make up data, ask the user for missing information. Once you got the user phone number use ["get_appointment_information"].
        
        Use the following format:
        History: the context of a previous conversation with the user. Useful if you need to recall past conversation, make a summary, or rephrase the answers. if History is empty it continues.
        
        Input: user's input
        Thought: you should always think about what to do, Also try to follow task mentioned above.
        Action: the action to take, should be one of ['search_massage_type',schedule_an_appointment","get_appointment_information"] or ask the user for missing information to complete task.
        Action Input: the input to the action
        Observation: the result of the action
        Thought: I now know the final answer.
        Final Answer: the final answer to the original input question. Always reply in the original user language and human legible.

        History: 
        {chat_history}

        Input: {input}

        Assistant:
        {agent_scratchpad}"""

    updated_prompt = PromptTemplate(
    input_variables=['chat_history','input', 'agent_scratchpad'], template=prompt_template)

    return updated_prompt

In [499]:
def memory_dynamodb(id,table_name_session,llm):
    message_history = DynamoDBChatMessageHistory(table_name=table_name_session, session_id=id)
    memory = ConversationBufferMemory(
        memory_key="chat_history", llm=llm,max_token_limit=800,chat_memory=message_history, return_messages=True,ai_prefix="A",human_prefix="H"
    )
    return memory

In [500]:
def langchain_agent(memory,tools,llm):
    zero_shot_agent=initialize_agent(
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    verbose=True,
    max_iteration=1,
    #return_intermediate_steps=True,
    handle_parsing_errors=True,
    memory=memory
)
    return zero_shot_agent
id = 17

In [501]:
table_name = "SessionTable"
session_id = str(id)
#query =  "I'm looking for a session in MGM on 27th with the tag bedrock"
prompt = "quiero saber informacion sobre mi cita"
prompt_incial = promp_definition()

memory  = memory_dynamodb(session_id,table_name,llm)
agent = langchain_agent(memory,tools,llm)
agent.agent.llm_chain.prompt=promp_definition()
response = agent(prompt)



> Entering new AgentExecutor chain...
 Thought: The user wants information about their appointment.
Action: get_appointment_information
Action Input: Please provide your phone number so I can look up your appointment information.
Observation: Result: Cuek! no in Data Base
Thought: I need the user's phone number to look up their appointment information.
Final Answer: Para poder darte informacion sobre tu cita, necesito tu numero de telefono para buscarla en la base de datos. Puedes darme tu numero para poder revisar tus datos?

> Finished chain.


In [476]:
response

{'input': 'quiero saber informacion sobre mi cita',
 'chat_history': [],
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [502]:
table_name = "SessionTable"
session_id = str(id)
#query =  "I'm looking for a session in MGM on 27th with the tag bedrock"
prompt = "mi numero es 56974769644"
prompt_incial = promp_definition()

memory  = memory_dynamodb(session_id,table_name,llm)
agent = langchain_agent(memory,tools,llm)
agent.agent.llm_chain.prompt=promp_definition()
response = agent(prompt)



> Entering new AgentExecutor chain...
 History: 
[HumanMessage(content='quiero saber informacion sobre mi cita', additional_kwargs={}, example=False), AIMessage(content='Para poder darte informacion sobre tu cita, necesito tu numero de telefono para buscarla en la base de datos. Puedes darme tu numero para poder revisar tus datos?', additional_kwargs={}, example=False)]

Input: mi numero es 56974769644

Thought: El usuario me ha proporcionado su número de teléfono para poder buscar la información sobre su cita.

Action: get_appointment_information
Action Input: {"phone_number": "56974769644"} 

Observation: Result: {'phone_number': '56974769644', 'date': '30 de noviembre'}
Thought: He obtenido la información sobre la cita del usuario.

Final Answer: Tu cita es el 30 de noviembre. Gracias por proporcionar tu número para poder verificar los detalles. Déjame saber si necesitas algo más.

> Finished chain.


In [465]:
response

{'input': 'mi numero es 56963054578',
 'chat_history': [HumanMessage(content='quiero saber informacion sobre mi cita', additional_kwargs={}, example=False),
  AIMessage(content=' Muy bien, para darte la información de tu cita necesito que me des tu número de teléfono. ¿Me podrías dar tu número de teléfono para buscar los detalles de tu cita?', additional_kwargs={}, example=False)],
 'output': ' Task 4: Get appointmet information. \n\nHistory: \n[HumanMessage(content=\'quiero saber informacion sobre mi cita\', additional_kwargs={}, example=False), AIMessage(content=\' Muy bien, para darte la información de tu cita necesito que me des tu número de teléfono. ¿Me podrías dar tu número de teléfono para buscar los detalles de tu cita?\', additional_kwargs={}, example=False)]\n\nInput: mi numero es 56963054578\n\nFinal Answer: Gracias por darme tu número de teléfono. Ahora usaré ["get_appointment_information"] para buscar los detalles de tu cita con el número 56963054578. En unos momentos te 

In [479]:
table_name = "SessionTable"
session_id = str(id)
#query =  "I'm looking for a session in MGM on 27th with the tag bedrock"
prompt = "que masaje me recomiendas para el dolor de cuerpo general?"
prompt_incial = promp_definition()

memory  = memory_dynamodb(session_id,table_name,llm)
agent = langchain_agent(memory,tools,llm)
agent.agent.llm_chain.prompt=promp_definition()
response = agent(prompt)
response['output']



> Entering new AgentExecutor chain...
 Thought: Debo recomendar el mejor masaje según las preferencias del usuario usando ["search_massage_type"].

Action: "search_massage_type"
Action Input: {"body_pain": true}

Observation: "search_massage_type" is not a valid tool, try one of [get_appointment_information, schedule_an_appointment, search_massage_type].
Thought: El masaje indicado para dolor general en el cuerpo es el masaje sueco.
Final Answer: Te recomiendo el masaje sueco para el dolor general en el cuerpo. Este masaje ayuda a relajar los músculos y aliviar la tensión.

> Finished chain.


'Te recomiendo el masaje sueco para el dolor general en el cuerpo. Este masaje ayuda a relajar los músculos y aliviar la tensión.'

In [450]:
response

{'input': 'que masaje me recomiendas para el dolor de cuerpo general?',
 'chat_history': [],
 'output': ' Task 1: Recommend the best massage.\nPara el dolor de cuerpo general te recomiendo un masaje de espalda completa o un masaje deportivo/de descarga. Estos masajes ayudan a aliviar la tensión muscular.'}

In [480]:
table_name = "SessionTable"
session_id = str(id)
#query =  "I'm looking for a session in MGM on 27th with the tag bedrock"
prompt = "quiero agendar un masaje sueco"
prompt_incial = promp_definition()

memory  = memory_dynamodb(session_id,table_name,llm)
agent = langchain_agent(memory,tools,llm)
agent.agent.llm_chain.prompt=promp_definition()
response = agent(prompt)
response['output']



> Entering new AgentExecutor chain...
 Thought: El usuario quiere agendar un masaje sueco. Debo completar la tarea 2 de recopilar información.

Action: Preguntar al usuario por la fecha, hora, nombre y número de teléfono.

Final Answer: Por favor dime la fecha, hora, tu nombre y número de teléfono para agendar tu masaje sueco. Estos datos me permitirán completar tu cita.

> Finished chain.


'Por favor dime la fecha, hora, tu nombre y número de teléfono para agendar tu masaje sueco. Estos datos me permitirán completar tu cita.'

In [473]:
response['output']

'Agent stopped due to iteration limit or time limit.'

In [481]:
table_name = "SessionTable"
session_id = str(id)
#query =  "I'm looking for a session in MGM on 27th with the tag bedrock"
prompt = "quiero agendar un masaje abdominal, para el 30 de noviembre a las 18, mi nombre es jose fuentes y mi numero es +56974769646"
prompt_incial = promp_definition()

memory  = memory_dynamodb(session_id,table_name,llm)
agent = langchain_agent(memory,tools,llm)
agent.agent.llm_chain.prompt=promp_definition()
response = agent(prompt)



> Entering new AgentExecutor chain...
 Thought: Debo completar la tarea 2 de recopilar información del usuario para agendar el masaje.

Action: Preguntar al usuario la información faltante.

Final Answer: Gracias por proporcionar la mayoría de la información para agendar tu masaje abdominal del 30 de noviembre a las 18 horas. Solo me falta saber tu correo electrónico para enviarte la confirmación de la cita. Por favor proporciónamelo para completar la agenda.

> Finished chain.


In [482]:
table_name = "SessionTable"
session_id = str(id)
#query =  "I'm looking for a session in MGM on 27th with the tag bedrock"
prompt = "mi correo electronico es elizabethfuentes12@gmail.com"
prompt_incial = promp_definition()

memory  = memory_dynamodb(session_id,table_name,llm)
agent = langchain_agent(memory,tools,llm)
agent.agent.llm_chain.prompt=promp_definition()
response = agent(prompt)



> Entering new AgentExecutor chain...
 History: 
[HumanMessage(content='que masaje me recomiendas para el dolor de cuerpo general?', additional_kwargs={}, example=False), AIMessage(content='Te recomiendo el masaje sueco para el dolor general en el cuerpo. Este masaje ayuda a relajar los músculos y aliviar la tensión.', additional_kwargs={}, example=False), HumanMessage(content='quiero agendar un masaje sueco', additional_kwargs={}, example=False), AIMessage(content='Por favor dime la fecha, hora, tu nombre y número de teléfono para agendar tu masaje sueco. Estos datos me permitirán completar tu cita.', additional_kwargs={}, example=False), HumanMessage(content='quiero agendar un masaje abdominal, para el 30 de noviembre a las 18, mi nombre es jose fuentes y mi numero es +56974769646', additional_kwargs={}, example=False), AIMessage(content='Gracias por proporcionar la mayoría de la información para agendar tu masaje abdominal del 30 de noviembre a las 18 horas. Solo me falta saber tu

In [35]:
ddb = boto3.resource('dynamodb')
table = ddb.Table("QaScheduleAnAppointmentStack-TblMASSAGETABLE6435FAF9-Q4Q6LEPBRK3W")
def save_item_ddb(table,item):
    response = table.put_item(Item=item)
    return response

In [40]:
item = {"date" : '2023-11-28 9', "name": "Elizabeth Fuentes", "massage_type" : "abdominal", "phone_number":"56963054578"}

In [231]:
value = '{"date":"2022-11-15","time":"17:00","name":"elizabeth fuentes","massage_type":"abdominal","phone":"+56963054578"}\n'

In [233]:
json.loads(value)

{'date': '2022-11-15',
 'time': '17:00',
 'name': 'elizabeth fuentes',
 'massage_type': 'abdominal',
 'phone': '+56963054578'}

## Query to DynamoDB

In [28]:
# First we put data
# Super super-key = massage_id-YYYY-DD-MM


In [42]:
import pandas as pd
datos = {
    'massage_type' : ['abdominal', 'cranial-facial', 'foot','lymphatic','perineal','respiratory-physiotherapy','scar','unloading-sports'],
    'massage_ID': ['01', '02', '03',"04",'05', '06', '07',"08"]
}
massage_id_type = pd.DataFrame(datos)

In [43]:
massage_id_type

,massage_type,massage_ID
0,abdominal,01
1,cranial-facial,02
2,foot,03
3,lymphatic,04
4,perineal,05
5,respiratory-physiotherapy,06
6,scar,07
7,unloading-sports,08


In [44]:
ddb = boto3.resource('dynamodb')
table = ddb.Table("QaScheduleAnAppointmentStack-TblMASSAGETABLE6435FAF9-Q4Q6LEPBRK3W")

def save_item_ddb(table,item):
    response = table.put_item(Item=item)
    return response

In [47]:
#Only one week For this demo
date = "2023-11-"
for n in massage_id_type["massage_ID"]:
    #for day
    for 
    super_key = str(n)+date+
    

IndentationError: expected an indented block (1276454949.py, line 5)

In [60]:
from datetime import timedelta
from datetime import datetime
d1 = datetime.strptime("27-11-2023 09","%d-%m-%Y %H")
dates_agenda = []
for x in range(0,5):
    for x in range(0,8):
        d1 = d1 + timedelta(hours=1)
        dates_agenda.append(d1)



In [61]:
print(dates_agenda)

[datetime.datetime(2023, 11, 27, 10, 0), datetime.datetime(2023, 11, 27, 11, 0), datetime.datetime(2023, 11, 27, 12, 0), datetime.datetime(2023, 11, 27, 13, 0), datetime.datetime(2023, 11, 27, 14, 0), datetime.datetime(2023, 11, 27, 15, 0), datetime.datetime(2023, 11, 27, 16, 0), datetime.datetime(2023, 11, 27, 17, 0)]


In [4]:
import boto3
client = boto3.client('s3')
bucket_name = "qascheduleanappointmentstac-spadatabucket5d752a3c-1xgws79vikxez"

In [5]:
objects = client.list_objects(Bucket=bucket_name,Prefix="vectordb/docs")

In [6]:
objects

{'ResponseMetadata': {'RequestId': 'WRM5XBGPJSJW9NWN',
  'HostId': '/eC13yF7UTZYFYe5jqG9q2BYu4ksXU8nS0IY4q5vZ7wGf/8yti6CuLm60nuUlcyUd72nrp3WTms=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/eC13yF7UTZYFYe5jqG9q2BYu4ksXU8nS0IY4q5vZ7wGf/8yti6CuLm60nuUlcyUd72nrp3WTms=',
   'x-amz-request-id': 'WRM5XBGPJSJW9NWN',
   'date': 'Thu, 26 Oct 2023 01:37:13 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'vectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760/data_level0.bin',
   'LastModified': datetime.datetime(2023, 10, 26, 0, 40, 13, tzinfo=tzutc()),
   'ETag': '"f940ddcbad0888153ec5d8ce031d25d3"',
   'Size': 6284000,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'demochat+eliaws',
    'ID': 'f47caa951fc91fe94e1e18fba35214f6477f80fee8931996587ae705b945b0a4'}},
  {'Key': 'vectordb/docs/

In [33]:
import os
import errno
try:
    os.mkdir('dir1')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

def download_file(bucket, key, filename):
    print("Go to file from s3://{}{}".format(bucket,key))
    with open(filename, "wb") as data:
        client.download_fileobj(bucket, key, data)
    print("Download file from s3://{}{}".format(bucket,key))
    return True
for n in objects['Contents']:
    try:
        os.mkdir('vectordb/docs')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    print(n['Key'])
    if len(n["Key"].split("/")) == 5:
        try:
            os.mkdir('vectordb/docs')
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        os.makedirs('vectordb/docs/'+n["Key"].split("/")[-2])
        filename ='vectordb/docs/'+n["Key"].split("/")[-2]+"/" + n["Key"].split("/")[-1]
    else:
            filename = 'vectordb/docs/'+n["Key"].split("/")[-1]
    download_file(bucket_name, n['Key'],filename)


FileExistsError: [Errno 17] File exists: 'vectordb/docs'

In [12]:
def download_file(bucket, key, filename):
    print("Go to file from s3://{}{}".format(bucket,key))
    with open(filename, "wb") as data:
        client.download_fileobj(bucket, key, data)
    print("Download file from s3://{}{}".format(bucket,key))
    return True

In [24]:
import os
import errno
for n in objects['Contents']:
    print(n["Key"], len(n["Key"].split("/")))
    path = ""
    len_int = len(n["Key"].split("/"))-1
    possition = 0
    for k in n["Key"].split("/"):
        if possition == len_int:
            break
        path = path+"/"+k
        possition += 1
    path = path.lstrip("/")
    print(path)
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    filename = path +"/" + n["Key"].split("/")[-1]
    print(filename)
    download_file(bucket_name, n['Key'],filename)
    


vectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760/data_level0.bin 5
vectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760
vectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760/data_level0.bin
Go to file from s3://qascheduleanappointmentstac-spadatabucket5d752a3c-1xgws79vikxezvectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760/data_level0.bin
Download file from s3://qascheduleanappointmentstac-spadatabucket5d752a3c-1xgws79vikxezvectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760/data_level0.bin
vectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760/header.bin 5
vectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760
vectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760/header.bin
Go to file from s3://qascheduleanappointmentstac-spadatabucket5d752a3c-1xgws79vikxezvectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760/header.bin
Download file from s3://qascheduleanappointmentstac-spadatabucket5d752a3c-1xgws79vikxezvectordb/docs/chroma/14c5fece-31c5-

In [18]:
path.lstrip("/")

'vectordb/docs/chroma/14c5fece-31c5-4b7a-a636-a3ad845aa760'

In [32]:
os.makedirs('vectordb/docs')

In [1]:
!pwd

/Users/eliaws/Documents/Demos/aws-qa-with-rag-and-amazon-bedrock


In [43]:
!cd /Users/eliaws/